# Visualization
Roboquant Notebooks come out of the box with support for several types of charts. They will help to better understand what is going on during a run and evaluate the performance of the underlying strategy. 

Under the hood roboquant leverages the excellent [Apache ECharts](https://echarts.apache.org/en/index.html)  visualization library to render the charts in Notebooks. ECharts was selected because:

* [x]  it has a very feature rich set of charts
* [x]  it can handle large data sets
* [x]  easy to add some interactive behavior like filtering and selection
* [x]  Apache opensource project and license


<div class="alert alert-block alert-info">
    <b>Tips: </b>
<ul>
    <li>
If you double click on a chart it will resize the chart to fit the cell output space. This comes in handy if you have resized the workspace or browser window.
    </li><li>
Within Jupyter Lab, roboquant will by default adapt to the used theme. So if you select the dark theme in the settings menu of Jupyter Lab, the charts rendered by roboquant will use an appropiate color scheme and everything will look great.
     </li><li>
Roboquant auto-detects the environment it is running in, either Jupyter Classic or Jupyter Lab. However this detection is not 100% accurate. But you can always invoke *Ouput.lab()* or *Output.classic()*, so roboquant knows the environment and will generate the corresponding output. 
     </li><li>
The best experience is with Jupyter Lab. But if you have to use the classic Jupyter Notebook environment, it is best to use a Chrome based browser or Safari since Firefox is known to have to some compatibility issues that can cause charts not to render.
     </li>
</ul>
</div>

In [ ]:
%use @http://roboquant.org/roboquant.json
Welcome()

# Price Feeds

A feed represents a stream of events you can use in your strategies. There are several type of feeds supported, like the one for Avro files used here. 

The file using in this notebook contains 5 years of S&P 500 daily prices (OHLCV). The Avro file format allows to store, access and compress data efficiently and works great for historic prices used in back-testing. This feed implementation also has low memory consumption since it only loads data when required. If you want to find out more about Avro files, check out their [documentation pages](https://avro.apache.org/docs/current/index.html).


Roboquant also supports plain CSV files as well as several third party proviers like Yahoo Finance, IEX Cloud, Binance, Alpaca and many others. More detailed examples can be found in the *Feeds Sample Notebook* and the roboquant documentation.

In [ ]:
val feed = AvroFeed.sp500()

// Show the first 10 assets in this feed
feed.assets.take(10).summary()

## Asset Performance Chart
This chart enables you to see which assets got most traded and how they performed:

1. The volume decides how big the asset is plotted. The volume is the base currency time the number units traded.
2. The return (in %) decides the color that is used. 

In [ ]:
AssetPerformanceChart(feed, Timeframe.fromYears(2017, 2018))

## Price Chart
The PriceChart replay the stream of events found in a feed and plots the prices it finds for a particular asset. In this example we'll find Apple stock (symbol name is "AAPL") and plot it.

In [ ]:
val apple = feed.find("AAPL")
PriceChart(feed, apple)

## PriceBar Chart
Roboqant isn't a technical analysis tool where you draw some support lines on a candlestick chart. But that doesn't mean candlesticks cannot come in handy. For example to validate if the data in a feed is not corrupted or has some other strange anomalies.

In [ ]:
PriceBarChart(feed, apple)

## Price Correlation Chart
This chart shows the correlation matrix of the prices of a number of assets within a feed. In this example we select some well known stocks.

In [ ]:
val assets = feed.assets.findBySymbols("AAPL", "GOOGL", "F", "BAC", "T", "MSFT", "CAT", "JPM", "IBM", "FB", "UBER", "PFE")
PriceCorrelationChart(feed, assets)

# Account
We run a simple strategy over the feed we just created to get some results to use for the charts. For this notebook we use the EMA crossover strategy with its default settings and capture the AccountSummary metrics. After the run has finsihed, the account contains the latest state. There are several charts that provide some insights into this. 

In [ ]:
val strategy = EMACrossover()
val roboquant =  Roboquant(strategy, AccountSummary())
roboquant.run(feed)

## Asset Allocation Charts
Lets see what assets are in our portfolio at the end of the run. We can plot only the assets (pie chart) or plot the assets grouped per asset-class (sunburst chart). Optionally we can also say if the cash positions in the account should be included in the chart.

In [ ]:
val account = roboquant.broker.account
AssetAllocationChart(account, includeCash = false, includeAssetClass=false)

In [ ]:
AssetAllocationChart(account, includeCash = true, includeAssetClass=true)

In [ ]:
// You can ofcourse also just print a summary
account.portfolio.summary()

## Trade Chart
Another important aspect is to see which trades has been executed as part of the run and how they performed. 

In [ ]:
TradeChart(account.trades.outliers())

In [ ]:
val chart = TradeChartByAsset(account.trades[0..50])
chart.height = 1000
chart

In [ ]:
// Lets plot biggest loosing trades
val trades = account.trades.filter { it.pnl < -2000 }
TradeChart(trades, aspect = "cost") 

In [ ]:
PriceBarChart(feed, apple, account.trades)

##  Order Chart 
We can also view the orders that were created as part of the run. This time we don't plot the orders of all assets, but only of AMD.

In [ ]:
val orders = account.closedOrders.filter { it.asset.symbol == "AMD" }
OrderChart(orders) 

# Metrics

Besides the charts for account related data, we can also plot the metrics that where captured during the run. Where an Account contains only the end state after the run has finished, metrics are catured during each step of the run. This allows for charts that provide more insights into different periods of the run. We used the AccountSummary metric that keeps track of how our account performs during the back-test. We use the account value (= cash + positions), for this sample

In [ ]:
val logger = roboquant.logger

// What metrics are available
logger.metricNames.summary("Metrics Names")

## Metric Chart

In [ ]:
val data = logger.getMetric("account.equity")
MetricChart(data)

## Metric Box Chart
If you want to inspect how volatile the performance of a strategy is, the metric box chart comes in handy. It provides insights into how a metric is distributed over a period of time. If provides the following info per box:

- The minimum
- low percentile (default is 25.0)
- mid percentile (default is 50.0, aka median)
- high percentile (default is 75.0)
- the maximum 

So in this case it shows these statsistics of the **account value change** based on the monthly aggregate of daily values.

In [ ]:
MetricBoxChart(data.perc())

In [ ]:
MetricBoxChart(data.diff(), period=ChronoUnit.YEARS, lowPercentile = 10.0, highPercentile = 90.0)

## Metric Histogram
If we want to see the distribution of the metric values, we can use the MetricHistogram chart. Besides the metric data that needs to be plot, you can also specify the number of bins. On the x-axis the upperbound of each bin is displayed and on the y-axis how many oberservations fit in that bin. 

In [ ]:
MetricHistogram(data.diff(), 15)

## Calender Chart
When evaluating the performance of a strategy, it is sometimes required to find out when the strategy was performing good and bad. A calendar charts allows you to plot a metric value per day, so it is easy to find out which days to further investigate. The slider at top of the chart allows for filtering only those days that are of interest based on their value.

In the example below we plot the daily account change and loss to see when the strategy performed good (green) and when bad (red). It also demonstrates how to display metrics only for a limited timeframe.

In [ ]:
val timeframe = Timeframe.fromYears(2016, 2018)
val data2 = data.filter { timeframe.contains(it.info.time) }
CalendarChart(data2.diff())

# Multi-run tests
A good way to back-test your strategy is to run it multiple times over different time-frames. This section shows how charts that come in handy to better understand the results of those type of back-tests. 

## Walk-forward
A commonly used approach in back-testing is the so called walk-forward. The historic data is split in a number of equal periods and then the strategy is run for each period seperately. This shows how the strategy would have performed during these different periods and possible different regimes.

In the following code we split the data in periods of 1 year and for each period run it. We then plot again the account value as observed during each run.

In [ ]:
roboquant.reset()

feed.split(1.years).forEach { 
    roboquant.run(feed, it)   
}

val data = roboquant.logger.getMetric("account.equity")
MetricChart(data)

## Random sampling
Another great way to back-test your strategy is to run it for a fixed duration at many random starting points in time. This gives you a good insight what could be a good and bad outcome of your strategy. The following piece of code runs the strategy 25 times, each time for the duration of 250 trading days. Feel free to change the number of runs from 25 to a higher value like 50 or 100. It is kept small so that also people with older or less powerfull computers don't have to wait too long.

In this case we plot the *account value* (= cash + positions) again and you can see there are times we do well but there are also periods when we loose money.

In [ ]:
roboquant.reset()

repeat(25) {
    val tf = feed.sample(250) // get a sample with 250 trading 
    roboquant.run(feed, tf) 
}

In [ ]:
val data = roboquant.logger.getMetric("account.equity")
MetricChart(data, useTime=false)

If you want to see the same data, but this time plotted in time, you can use the same MetricChart class and provide the *useTime=true* parameter. This shows in which periods the strategy is performing well and in which periods it is struggeling. It is similar to the walk-forward approach, but provides at each moment in time multiple samples.

In [ ]:
MetricChart(data, useTime=true)

# Extra

## Displaying multiple charts

By using the *chart.render()* method you can display multiple charts in the output section of a single cell. The following code selects 2 random assets from the feed and renders a candlestick chart for each one of them. It also demonstrates the excellent performance of the ECharts library, since every chart contains many data points.

In [ ]:
val assets = feed.assets.take(2)
for (asset in assets) PriceBarChart(feed, asset).render()

Another example, lets see how the correlation between 5 random assets differs over timeframes of maximum 2 years. If you see empty matrix cells, it is because there are not enough observations to calculate a correlation for those two assets.

In [ ]:
val assets = feed.assets.take(5)
feed.timeframe.split(2.years).forEach { 
    val chart = PriceCorrelationChart(feed, assets, it)
    chart.height = 300
    chart.render()
}